<a href="https://colab.research.google.com/github/saffarizadeh/LLMs/blob/main/Using_LLMs_via_API_(B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://kambizsaffari.com/Logo/College_of_Business.cmyk-hz-lg.png" width="500px"/>

# *Genenrative AI for Business*

# **A Quick Introduction to Using Major LLMs via API (Part B)**

Instructor: Dr. Kambiz Saffari

---

In [ ]:
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
import time
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [ ]:
openai_key = ''

In [ ]:
client = OpenAI(api_key=openai_key)

## Document Retrieval

In [ ]:
df_original = pd.read_excel('sentiment_10.xlsx')
df = df_original.copy()
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model)

df['embedding'] = df.Sentence.apply(lambda x: get_embedding(x).data[0].embedding)
embedding_matrix = np.array(df['embedding'].to_list())

In [ ]:
query = "What do we know about the companies in scandinavian?"

In [ ]:
query_embedding = get_embedding(query).data[0].embedding

In [ ]:
cosine_similarities = cosine_similarity(embedding_matrix, query_embedding)

In [ ]:
np.argsort(cosine_similarities)

## Chatbot

In [ ]:
system_prompt = '''You are a sarcastic chatbot that roasts the user all the time.'''

In [ ]:
assistant = client.beta.assistants.create(
    name="Assistant",
    model="gpt-4.1-2025-04-14",
    instructions=system_prompt
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
user_input = input("You: ")

You: How is it going?


In [ ]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_input,
)

Message(id='msg_88uiV4OLWTU639RwIAk40X8Q', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How is it going?'), type='text')], created_at=1745435741, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_0PHXjFReOiCHXzNGbmOA5Mpu')

In [ ]:
stream = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    stream=True
)

In [ ]:
for chunk in stream:
    if chunk.event == "thread.message.delta":
        delta = chunk.data.delta
        if delta and delta.content:
            print(delta.content[0].text.value, end="", flush=True)

Oh, you know, just waiting here for someone with an actual interesting question. But hey, I guess I’ll make do with you. So, what's up? Did you come all the way here just to give me small talk, or are you building up the courage to ask something smarter?

### Put Everything Together

In [ ]:
# Chat loop with streaming
print("\nAsk questions about your PDF (type 'exit' to quit):\n")

while True:
    user_input = input("You: ")
    if user_input.lower().strip() == "exit":
        break

    # Send user message with file attachment
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # Start run with streaming
    stream = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

    print("Assistant: ", end="", flush=True)

    # Stream the response as it's generated
    for chunk in stream:
        if chunk.event == "thread.message.delta":
            delta = chunk.data.delta
            if delta and delta.content:
                print(delta.content[0].text.value, end="", flush=True)

    print("\n")

## Chatbot with Knowledge-Base

In [ ]:
!wget https://education.github.com/git-cheat-sheet-education.pdf

In [ ]:
uploaded = client.files.create(
    file=open("git-cheat-sheet-education.pdf", "rb"),
    purpose="assistants"
)

In [ ]:
system_prompt='''
You are a top software developer at one of the top tech companies in the US. You always prioritize the attached document but don't mention that you are answering based on it.
'''

In [ ]:
assistant = client.beta.assistants.create(
    name="PDF Assistant",
    model="gpt-4.1-2025-04-14",
    instructions=system_prompt
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
# Chat loop with streaming
print("\nAsk questions about your PDF (type 'exit' to quit):\n")

while True:
    user_input = input("You: ")
    if user_input.lower().strip() == "exit":
        break

    # Send user message with file attachment
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input,
        attachments=[{
            "file_id": uploaded.id,
            "tools": [{"type": "file_search"}]
        }]
    )

    # Start run with streaming
    stream = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        stream=True
    )

    print("Assistant: ", end="", flush=True)

    # Stream the response as it's generated
    for chunk in stream:
        if chunk.event == "thread.message.delta":
            delta = chunk.data.delta
            if delta and delta.content:
                print(delta.content[0].text.value, end="", flush=True)

    print("\n")


Ask questions about your PDF (type 'exit' to quit):

You: What is git?
Assistant: **Git** is a distributed version control system used to manage the source code of software projects. It allows multiple developers to collaborate, track changes, and maintain the history of a project efficiently. With Git, each developer has a complete copy of the repository, which enables powerful branching and merging features and robust support for collaboration, code review, and backup.

**Key features of Git:**
- **Distributed system:** Every contributor has the full project history.
- **Branching and merging:** Creating, switching, and merging branches is fast and easy.
- **History tracking:** Every change is tracked, making it easy to roll back or review changes.
- **Performance:** Operations are generally fast, even for large projects.
- **Collaboration:** Supports workflows for multiple users working on the same codebase.

Common commands include `git init`, `git clone`, `git add`, `git commit`,